In [4]:
import pandas as pd
from sqlalchemy import create_engine
from config import password
from sqlalchemy.sql import text as sa_text

rds_connection_string = f'postgres:{password}@localhost:5432/thor'
engine = create_engine(f'postgresql://{rds_connection_string}')

In [5]:
# display the table names in the thor database
engine.table_names()

['thor_wwii_master',
 'thor_wwii_weather',
 'thor_wwii_aircraft',
 'thor_wwii_station_location']

In [7]:
# truncate the tables to get rid of existing data
engine.execute(sa_text('''TRUNCATE TABLE thor_wwii_master CASCADE;''').execution_options(autocommit=True))
engine.execute(sa_text('''TRUNCATE TABLE thor_wwii_aircraft CASCADE;''').execution_options(autocommit=True))
engine.execute(sa_text('''TRUNCATE TABLE thor_wwii_station_location CASCADE;''').execution_options(autocommit=True))
engine.execute(sa_text('''TRUNCATE TABLE thor_wwii_weather CASCADE;''').execution_options(autocommit=True))

## Step 1. thor_wwii_aircraft ETL

In [8]:
# path to the thor_wwii_aircraft file
aircraft_csv = './Resources/THOR_WWII_AIRCRAFT_GLOSS.csv'


# load the THOR_WWII_AIRCRAFT_GLOSS.csv file into a df
aircraft_df = pd.read_csv(aircraft_csv)
aircraft_df.head()

,GLOSS_ID,AIRCRAFT,NAME,FULL_NAME,AIRCRAFT_TYPE,HYPERLINK
0,1,A20,A20,Douglas A-20 Havoc,Boston Light Bomber / Night-Fighter,http://militaryfactory.com/aircraft/detail.asp...
1,2,A24,A24,Douglass A-24 Banshee,Dive Bomber / Reconnaissance,http://militaryfactory.com/aircraft/detail.asp...
2,3,A26,A26,Douglas A-26 Invader,Medium Bomber / Heavy Assault,http://militaryfactory.com/aircraft/detail.asp...
3,4,A36,A36,North American A-36 Apache (Invader),Ground Attack / Dive Bomber,http://militaryfactory.com/aircraft/detail.asp...
4,5,ALBA,Albacore,Fairey Albacore,Naval Torpedo Bomber,http://militaryfactory.com/aircraft/detail.asp...


In [9]:
# Create a new dataframe with the fields needed for the insert
new_aircraft_df = aircraft_df[['AIRCRAFT', 'NAME', 'FULL_NAME', 'AIRCRAFT_TYPE','HYPERLINK']].copy()
new_aircraft_df = new_aircraft_df.rename(columns={
                                                    'AIRCRAFT': 'aircraft', 
                                                    'NAME': 'name',
                                                    'FULL_NAME': 'full_name',
                                                    'AIRCRAFT_TYPE':'aircraft_type',
                                                    'HYPERLINK':'hyperlink'
})

new_aircraft_df.head()

,aircraft,name,full_name,aircraft_type,hyperlink
0,A20,A20,Douglas A-20 Havoc,Boston Light Bomber / Night-Fighter,http://militaryfactory.com/aircraft/detail.asp...
1,A24,A24,Douglass A-24 Banshee,Dive Bomber / Reconnaissance,http://militaryfactory.com/aircraft/detail.asp...
2,A26,A26,Douglas A-26 Invader,Medium Bomber / Heavy Assault,http://militaryfactory.com/aircraft/detail.asp...
3,A36,A36,North American A-36 Apache (Invader),Ground Attack / Dive Bomber,http://militaryfactory.com/aircraft/detail.asp...
4,ALBA,Albacore,Fairey Albacore,Naval Torpedo Bomber,http://militaryfactory.com/aircraft/detail.asp...


In [10]:
# load the data into the table
new_aircraft_df.to_sql(name='thor_wwii_aircraft', con=engine, if_exists='append', index=False)

In [11]:
# test that the data was loaded into the database
pd.read_sql_query('select count(*) as record_count from thor_wwii_aircraft', con=engine).head()

,record_count
0,52


## Step 2. thor_wwii_station_location ETL

In [12]:
# path to the thor_wwii_station_location file
station_csv = './Resources/Weather Station Locations.csv'

# load the Weather Station Locations.csv file into a df
station_df = pd.read_csv(station_csv)
station_df.head()

,WBAN,NAME,STATE/COUNTRY ID,LAT,LON,ELEV,Latitude,Longitude
0,33013,AIN EL,AL,3623N,00637E,611,36.383333,6.650000
1,33031,LA SENIA,AL,3537N,00037E,88,35.616667,0.583333
2,33023,MAISON BLANCHE,AL,3643N,00314E,23,36.716667,3.216667
3,33044,TELERGMA,AL,3607N,00621E,754,36.116667,6.416667
4,12001,TINDOUF,AL,2741N,00809W,443,27.683333,-8.083333


In [13]:
# Create a new dataframe with the fields needed for the insert
new_station_df = station_df[['WBAN', 'NAME', 'STATE/COUNTRY ID', 'ELEV','Latitude','Longitude']].copy()
new_station_df = new_station_df.rename(columns={
                                                    'WBAN': 'wban', 
                                                    'NAME': 'name',
                                                    'STATE/COUNTRY ID': 'state_country',
                                                    'ELEV':'elev',
                                                    'Latitude':'latitude',
                                                    'Longitude': 'longitude'
                            
})

new_station_df.head()

,wban,name,state_country,elev,latitude,longitude
0,33013,AIN EL,AL,611,36.383333,6.650000
1,33031,LA SENIA,AL,88,35.616667,0.583333
2,33023,MAISON BLANCHE,AL,23,36.716667,3.216667
3,33044,TELERGMA,AL,754,36.116667,6.416667
4,12001,TINDOUF,AL,443,27.683333,-8.083333


In [14]:
# load the data into the table
new_station_df.to_sql(name='thor_wwii_station_location', con=engine, if_exists='append', index=False)

In [15]:
# test that the data was loaded into the database
pd.read_sql_query('select count(*) as record_count from thor_wwii_station_location', con=engine).head()

,record_count
0,161


## Step 3. thor_wwii_weather ETL

In [16]:
# path to the thor_wwii_weather file
weather_csv = './Resources/Summary of Weather.csv'

# load the Weather Station Locations.csv file into a df
weather_df = pd.read_csv(weather_csv,low_memory=False)
weather_df.head()

,STA,Date,Precip,WindGustSpd,MaxTemp,MinTemp,MeanTemp,Snowfall,PoorWeather,YR,...,FB,FTI,ITH,PGT,TSHDSBRSGF,SD3,RHX,RHN,RVG,WTE
0,10001,1942-7-1,1.016,NaN,25.555556,22.222222,23.888889,0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10001,1942-7-2,0,NaN,28.888889,21.666667,25.555556,0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10001,1942-7-3,2.54,NaN,26.111111,22.222222,24.444444,0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10001,1942-7-4,2.54,NaN,26.666667,22.222222,24.444444,0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10001,1942-7-5,0,NaN,26.666667,21.666667,24.444444,0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Create a new dataframe with the fields needed for the insert
new_weather_df = weather_df[['STA', 'Date', 'PRCP', 'DR','SPD','MAX','MIN','MEA','SNF']].copy()
new_weather_df = new_weather_df.rename(columns={
                                                    'STA': 'sta',
                                                    'Date': 'date',
                                                    'PRCP': 'prcp',
                                                    'DR': 'dr',
                                                    'SPD':'spd',
                                                    'MAX':'max',
                                                    'MIN': 'min',
                                                    'MEA': 'mea',
                                                    'SNF': 'snf'
                            
})

new_weather_df.head()

,sta,date,prcp,dr,spd,max,min,mea,snf
0,10001,1942-7-1,0.04,NaN,NaN,78.0,72.0,75.0,0
1,10001,1942-7-2,0,NaN,NaN,84.0,71.0,78.0,0
2,10001,1942-7-3,0.1,NaN,NaN,79.0,72.0,76.0,0
3,10001,1942-7-4,0.1,NaN,NaN,80.0,72.0,76.0,0
4,10001,1942-7-5,0,NaN,NaN,80.0,71.0,76.0,0


In [18]:
# load the data into the table
new_weather_df.to_sql(name='thor_wwii_weather', con=engine, if_exists='append', index=False)

In [19]:
# test that the data was loaded into the database
pd.read_sql_query('select count(*) as record_count from thor_wwii_station_location', con=engine).head()

,record_count
0,161


### Step 4. thor_wwii_master ETL

In [20]:
# path to the thor_wwii_weather file
thor_csv = './Resources/THOR_WWII_DATA_CLEAN.csv'

# load the Weather Station Locations.csv file into a df
thor_df = pd.read_csv(thor_csv, encoding = "ISO-8859-1", low_memory=False)
thor_df.head()

,WWII_ID,MASTER_INDEX_NUMBER,MSNDATE,THEATER,NAF,COUNTRY_FLYING_MISSION,TGT_COUNTRY_CODE,TGT_COUNTRY,TGT_LOCATION,TGT_TYPE,...,CALLSIGN,ROUNDS_AMMO,SPARES_RETURN_AC,WX_FAIL_AC,MECH_FAIL_AC,MISC_FAIL_AC,TARGET_COMMENT,MISSION_COMMENTS,SOURCE,DATABASE_EDIT_COMMENTS
0,1,NaN,8/15/1943,MTO,12 AF,USA,13.0,ITALY,SPADAFORA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4285,20028.0,2/20/1945,PTO,5 AF,USA,NaN,PHILIPPINE ISLANDS,PUERTA PRINCESA,UNIDENTIFIED TARGET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,8/15/1943,MTO,12 AF,USA,13.0,ITALY,COSENZA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,8/15/1943,MTO,12 AF,USA,13.0,ITALY,GIOJA TAURO,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
4,8167,14639.0,2/23/1945,PTO,5 AF,USA,NaN,PHILIPPINE ISLANDS,BALETE PASS,WOODED AREA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# create a series of aircraft from new_aircraft_df to be used in the merge
aircraft_series = new_aircraft_df['aircraft']

# rename MDS to aircraft for the merge
thor_stage_df = thor_df.rename(columns={'MDS':'aircraft'})

# merge thor_stage_df with aircraft
thor_merge_df = thor_stage_df.merge(aircraft_series, on="aircraft", how="left", indicator=True)

# create a df with records where the aircraft column has a value in the aircraft_series
thor_clean_df = thor_merge_df[thor_merge_df["_merge"] == "both"]

In [22]:
# Create a new dataframe with the fields needed for the insert from the thor_clean_df
new_thor_df = thor_clean_df[['MSNDATE',
                        'THEATER',
                        'NAF',
                        'COUNTRY_FLYING_MISSION',
                        'SOURCE_LATITUDE',
                        'SOURCE_LONGITUDE',
                        'LATITUDE',
                        'LONGITUDE',
                        'aircraft',
                        'TGT_ID',
                        'TGT_INDUSTRY_CODE',
                        'TGT_LOCATION',                       
                        'TGT_TYPE',
                        'TGT_INDUSTRY',
                        'TGT_COUNTRY',
                        'TGT_PRIORITY',
                        'AC_ATTACKING',
                        'ALTITUDE',
                        'NUMBER_OF_HE',
                        'TYPE_OF_HE',
                        'LBS_HE',
                        'TONS_OF_HE',
                        'NUMBER_OF_IC',
                        'TYPE_OF_IC',
                        'LBS_IC',
                        'TONS_OF_IC',
                        'NUMBER_OF_FRAG',
                        'TYPE_OF_FRAG',
                        'LBS_FRAG',
                        'TONS_OF_FRAG',
                        'TOTAL_LBS',
                        'TOTAL_TONS',
                        'TAKEOFF_BASE',
                        'TAKEOFF_COUNTRY',
                        'TAKEOFF_LATITUDE',
                        'TAKEOFF_LONGITUDE',
                        'AC_LOST',
                        'AC_DAMAGED',
                        'AC_AIRBORNE',
                        'AC_DROPPING',
                        'TIME_OVER_TARGET',
                        'BDA',
                        'CALLSIGN',
                        'SPARES_RETURN_AC',
                        'WX_FAIL_AC',
                        'MECH_FAIL_AC',
                        'MISC_FAIL_AC',
                        'TARGET_COMMENT',
                        'MISSION_COMMENTS']].copy()

new_thor_df = new_thor_df.rename(columns={
                                            'MSNDATE': 'msndate',
                                            'THEATER': 'theater',
                                            'NAF':'naf',
                                            'COUNTRY_FLYING_MISSION':'country_flying_mission',
                                            'SOURCE_LATITUDE': 'source_latitude',
                                            'SOURCE_LONGITUDE': 'source_longitude',
                                            'LATITUDE': 'latitude',
                                            'LONGITUDE': 'longitude',
                                            'TGT_ID' : 'tgt_id',
                                            'TGT_INDUSTRY_CODE': 'tgt_industry_code',
                                            'TGT_LOCATION': 'tgt_location',
                                            'TGT_TYPE': 'tgt_type',
                                            'TGT_INDUSTRY' : 'tgt_industry',
                                            'TGT_COUNTRY': 'tgt_country',
                                            'TGT_PRIORITY': 'tgt_priority',
                                            'AC_ATTACKING': 'ac_attacking',
                                            'ALTITUDE': 'altitude',
                                            'NUMBER_OF_HE': 'number_of_he',
                                            'TYPE_OF_HE': 'type_of_he',
                                            'LBS_HE': 'lbs_he',
                                            'TONS_OF_HE': 'tons_of_he',
                                            'NUMBER_OF_IC': 'number_of_ic',
                                            'TYPE_OF_IC': 'type_of_ic',
                                            'LBS_IC': 'lbs_ic',
                                            'TONS_OF_IC': 'tons_of_ic',
                                            'NUMBER_OF_FRAG': 'number_of_frag',
                                            'TYPE_OF_FRAG': 'type_of_frag',
                                            'LBS_FRAG': 'lbs_frag',
                                            'TONS_OF_FRAG': 'tons_of_frag',
                                            'TOTAL_LBS': 'total_lbs',
                                            'TOTAL_TONS': 'total_tons',
                                            'TAKEOFF_BASE': 'takeoff_base',
                                            'TAKEOFF_COUNTRY': 'takeoff_country',
                                            'TAKEOFF_LATITUDE': 'takeoff_latitude',
                                            'TAKEOFF_LONGITUDE': 'takeoff_longitude',
                                            'AC_LOST': 'ac_lost',
                                            'AC_DAMAGED': 'ac_damaged',
                                            'AC_AIRBORNE': 'ac_airborne',
                                            'AC_DROPPING': 'ac_dropping',
                                            'TIME_OVER_TARGET': 'time_over_target',
                                            'BDA': 'bda',
                                            'CALLSIGN': 'callsign',
                                            'SPARES_RETURN_AC': 'spares_return_ac',
                                            'WX_FAIL_AC': 'wx_fail_ac',
                                            'MECH_FAIL_AC': 'mech_fail_ac',
                                            'MISC_FAIL_AC': 'misc_fail_ac',
                                            'TARGET_COMMENT': 'target_comment',
                                            'MISSION_COMMENTS': 'mission_comments'
                                            
                            
})

new_thor_df.head()

,msndate,theater,naf,country_flying_mission,source_latitude,source_longitude,latitude,longitude,aircraft,tgt_id,...,ac_dropping,time_over_target,bda,callsign,spares_return_ac,wx_fail_ac,mech_fail_ac,misc_fail_ac,target_comment,mission_comments
0,8/15/1943,MTO,12 AF,USA,38.2166667,15.3666667,38.220000,15.370000,A36,40675.0,...,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2/20/1945,PTO,5 AF,USA,945,11845,9.750000,118.750000,A20,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8/15/1943,MTO,12 AF,USA,3916N,1615,39.270000,16.250000,A36,9630.0,...,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8/15/1943,MTO,12 AF,USA,3826N,1554,38.430000,15.900000,A36,16140.0,...,15.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,2/23/1945,PTO,5 AF,USA,1605,12055,16.083333,120.916667,A20,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# load the data into the table - this takes a few minutes
new_thor_df.to_sql(name='thor_wwii_master', con=engine, if_exists='append', index=False)

In [24]:
# test that the data was loaded into the database
pd.read_sql_query('select count(*) as record_count from thor_wwii_master', con=engine).head()

,record_count
0,154806
